In [1]:
%load_ext autoreload
%autoreload 2
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
from benchmark.construct_threatrace_graph import *
from benchmark.threatrace import *
import logging 
from torch_geometric import seed_everything
import os
import configparser
from patricks_modul import PatricksLog
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')



patricks_log = PatricksLog('gcn_1_0')

Time:2024-01-09 15:52:34


In [2]:
project_dir = os.getcwd()
print("Current project directory: ", project_dir)
patricks_log.write_kv_to_file('Current project directory', project_dir)

Current project directory:  c:\Users\patri\Desktop\gnn_testenv\notebooks


In [3]:
# Read config file
config_dir = "../configs"
config_file_record_03 = config_dir + '/eng3/cadets/cadets_03_record.ini'
config_rec3 = configparser.ConfigParser()
config_rec3.read(config_file_record_03)

['../configs/eng3/cadets/cadets_03_record.ini']

In [4]:
# read tar file and extract to a folder + preprocess
process_darpa_data(config_rec3)


In [5]:
#setup and import data to mariaDB 
import_nodes_and_edges(config_rec3)

In [6]:
print(config_rec3['threatrace'])

<Section: threatrace>


In [7]:
own_timestamp_percent=20
patricks_log.write_kv_to_file('Test/Train split', own_timestamp_percent)
# create train and test data
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent, extension="20percentTrain") # 4min
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print('Data_train:')
print(data_train)
print('Data_train_x: ')
print(data_train['x'][:1])
print('Data_train_edge_index: ')
print(data_train['edge_index'][:2])
print('Verteilung Train', np.bincount(data_train['y']))
print('Data_test: ')
print(data_test)
print('Verteilung Test', np.bincount(data_test['y']))
patricks_log.write_div_to_file()
patricks_log.write_kv_to_file('Data_train size of x / features', str(list(data_train.x.size())))
patricks_log.write_kv_to_file('Data_train size of edge_index', str(list(data_train.edge_index.size())))
patricks_log.write_kv_to_file('Data_train size of y', str(list(data_train.y.size())))
patricks_log.write_kv_to_file('Data_train distribution of y', str(np.bincount(data_train['y'])))
patricks_log.write_kv_to_file('Data_train size of train_mask / test_mask', str(list(data_train.train_mask.size())) + str(list(data_train.test_mask.size())))
patricks_log.write_to_file('')
patricks_log.write_kv_to_file('Data_test size of x / features', str(list(data_test.x.size())))
patricks_log.write_kv_to_file('Data_test size of edge_index', str(list(data_test.edge_index.size())))
patricks_log.write_kv_to_file('Data_test size of y', str(list(data_test.y.size())))
patricks_log.write_kv_to_file('Data_test distribution of y', str(np.bincount(data_test['y'])))
patricks_log.write_kv_to_file('Data_test size of train_mask / test_mask', str(list(data_test.train_mask.size())) + str(list(data_test.test_mask.size())))
patricks_log.save_time_since_start()
patricks_log.write_div_to_file()

Data_train:
Data(x=[154960, 56], edge_index=[2, 1738773], y=[154960], train_mask=[154960], test_mask=[154960])
Data_train_x: 
tensor([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0.,     0.,     0.,     0.,   370.,     0.,     0.,     0.,     0.,
            41.,     0.,     0.,     0.,     0.,     0.,    51.,     0.,   205.,
             0.,  7894.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
             0., 10521.]])
Data_train_edge_index: 
tensor([[     0,      0,      1,  ...,     25,     25,     25],
        [     1,      1,      3,  ..., 154955, 154957, 154959]])
Verteilung Train [67717  8533 74229  2403  2039    39]
Data_test: 
Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])
Verteilung Test [133448  43080 374521  126

In [8]:
# create ground truth
gt_filename = 'gt.txt'
gt_load = True

if gt_load and os.path.exists(gt_filename):
    with open(gt_filename, 'r') as file:
        gt = [int(x) for x in file.readline().split()]
else:
    gt_file1 = config_rec3['Files']['ground_truth'] # -> special case for record3, because here 2 attacks are given 
    gt_file2 = config_rec3['Files']['ground_truth1'] # -> special case for record3, because here 2 attacks are given 
    gts = [gt_file1, gt_file2]
    gt = get_atk_nodes_for_evaluation(config_rec3, multi=True, gts=gts) # -> list with id´s [526, 524824, 537, 262769, ...]

    with open(gt_filename, 'w') as file:
        file.write(' '.join(map(str, gt)))

print(len(gt))

25049


In [9]:
# create threatrace GNN anomaly detection Pipeline 
ttp = ThreaTracePipeline(config_rec3, data_train, data_test, patricks_log) 
patricks_log.save_time_since_start()

Gesamtzeit seit beginn: 0 Stunden, 0 Minute, 4 Sekunden, 0.0 Millisekunden, 110.0 Mikrosekunden, 505.3424835205078 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 0 Minute, 2 Sekunden, 0.0 Millisekunden, 699.0 Mikrosekunden, 296.9512939453125 Nanosekunden


In [10]:
ttp.delete_old_models()
patricks_log.write_to_file('### PRETRAINING ### - wird nichtgesaved')
ttp.pretraining()
patricks_log.save_time_since_start()

Epoche: 1 Loss: 793.2549438476562 Accuracy: 0.45946695921528136 Time:2024-01-09 15:52:40
Epoche: 2 Loss: 1003.4878540039062 Accuracy: 0.4591701084150749 Time:2024-01-09 15:52:41
Epoche: 3 Loss: 475.0143127441406 Accuracy: 0.45980252968508 Time:2024-01-09 15:52:43
Epoche: 4 Loss: 779.1917114257812 Accuracy: 0.45997676819824473 Time:2024-01-09 15:52:44
Epoche: 5 Loss: 1524.9705810546875 Accuracy: 0.4597573567372225 Time:2024-01-09 15:52:46
Epoche: 6 Loss: 1171.436767578125 Accuracy: 0.4597250903458957 Time:2024-01-09 15:52:47
Epoche: 7 Loss: 1444.9498291015625 Accuracy: 0.4607769747031492 Time:2024-01-09 15:52:49
Epoche: 8 Loss: 1267.9912109375 Accuracy: 0.4741997934950955 Time:2024-01-09 15:52:50
Epoche: 9 Loss: 1398.3995361328125 Accuracy: 0.47492901393908105 Time:2024-01-09 15:52:51
Epoche: 10 Loss: 1315.1824951171875 Accuracy: 0.47735544656685597 Time:2024-01-09 15:52:53
Epoche: 11 Loss: 1337.125732421875 Accuracy: 0.49317243159525037 Time:2024-01-09 15:52:54
Epoche: 12 Loss: 542.447

In [11]:
patricks_log.write_to_file('\n### MULTI MODEL TRAINING ###')
ttp.multi_model_training()
patricks_log.save_time_since_start()

New Round
falseClassified
[0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 19, 22, 23, 25, 27, 30, 31, 33, 34, 36, 37, 38, 46, 62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 76, 78, 80, 101, 102, 103, 105, 106, 108, 112, 114, 116, 119, 120, 122, 123, 124, 125, 129, 130, 163, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 189, 190, 191, 192, 195, 203, 216, 217, 219, 220, 221, 222, 224, 225, 226, 227, 229, 230, 231, 232, 268, 269, 273, 276, 277, 278, 281, 282, 283, 284, 288, 289, 291, 292, 294, 295, 296, 326, 327, 329, 330, 331, 332, 333, 352, 359, 362, 363, 364, 365, 366, 367, 371, 372, 373, 374, 375, 377, 378, 379, 380, 381, 382, 383, 385, 387, 388, 389, 391, 392, 393, 394, 395, 398, 399, 400, 401, 402, 403, 404, 406, 415, 430, 432, 433, 460, 461, 491, 492, 498, 499, 502, 503, 506, 507, 508, 509, 512, 513, 516, 517, 526, 527, 528, 529, 530, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 560, 562, 565, 566, 633, 

In [12]:
print("##############################################################################################################")
print("own evaluation")
print("##############################################################################################################")
patricks_log.write_to_file('\n### OWN EVALUATION ### - Jedes Model mit dem Test Datensatz testen')
ttp.reinit_test_data() 
ttp.test_model_performance(gt) 
patricks_log.save_time_since_start()

##############################################################################################################
own evaluation
##############################################################################################################
Model_num: 0  Accuracy:0.8052  false_classified:111879  true_classified:462459
- false classified Verteilung [20104 43048 25572 12656 10377   122]
- true classified Verteilung [113344     32 348949      1    133]
Unique Count of pred: (tensor([False,  True]), tensor([462459, 111879]))
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 111879/111879 [00:30<00:00, 3707.66it/s]


{'fn': 103, 'fp': 7, 'tn': 549282, 'tp': 24946}
Precision:  0.9997194726085
Recall:  0.9958880594035651
F-Score:  0.9978000879464763
Accuracy:  0.99980847514878

Model_num: 1  Accuracy:0.1755  false_classified:473536  true_classified:100802
- false classified Verteilung [ 62698  15853 371698  12655  10510    122]
- true classified Verteilung [70750 27227  2823     2]
Unique Count of pred: (tensor([False,  True]), tensor([509557,  64781]))
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 64781/64781 [00:18<00:00, 3491.26it/s]


{'fn': 13002, 'fp': 3, 'tn': 549286, 'tp': 12047}
Precision:  0.9997510373443901
Recall:  0.4809373627689709
F-Score:  0.6494514676506821
Accuracy:  0.9773565391807612

Model_num: 2  Accuracy:0.0211  false_classified:562210  true_classified:12128
- false classified Verteilung [133051  41741 366699  10116  10481    122]
- true classified Verteilung [ 397 1339 7822 2541   29]
Unique Count of pred: (tensor([False,  True]), tensor([518405,  55933]))
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


...

{'fn': 23390, 'fp': 1, 'tn': 549288, 'tp': 1659}
Precision:  0.9993975903613855
Recall:  0.06623018882989315
F-Score:  0.12422778837427868
Accuracy:  0.9592731109555697

Model_num: 36  Accuracy:0.0150  false_classified:565751  true_classified:8587
- false classified Verteilung [131002  41928 371078  12050   9571    122]
- true classified Verteilung [2446 1152 3443  607  939]
Unique Count of pred: (tensor([False,  True]), tensor([563637,  10701]))
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 10701/10701 [00:03<00:00, 3366.31it/s]


{'fn': 23429, 'fp': 1, 'tn': 549288, 'tp': 1620}
Precision:  0.9993830968537323
Recall:  0.06467324044872025
F-Score:  0.12148481438678217
Accuracy:  0.9592052066901372

Model_num: 37  Accuracy:0.0179  false_classified:564063  true_classified:10275
- false classified Verteilung [131521  39702 372789  11944   7985    122]
- true classified Verteilung [1927 3378 1732  713 2525]
Unique Count of pred: (tensor([False,  True]), tensor([564286,  10052]))
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 10052/10052 [00:03<00:00, 3321.55it/s]


{'fn': 23453, 'fp': 1, 'tn': 549288, 'tp': 1596}
Precision:  0.9993738259235442
Recall:  0.06371511836799847
F-Score:  0.11979283943930494
Accuracy:  0.9591634194498709

Unique Count of data_flow.y:  (tensor([0, 1, 2, 3, 4, 5]), tensor([133448,  43080, 374521,  12657,  10510,    122]))
Gesamtzeit seit beginn: 0 Stunden, 41 Minute, 44 Sekunden, 0.0 Millisekunden, 957.0 Mikrosekunden, 561.492919921875 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 34 Minute, 12 Sekunden, 0.0 Millisekunden, 380.0 Mikrosekunden, 225.65841674804688 Nanosekunden


In [13]:
print("##############################################################################################################")
print("0 HOP")
patricks_log.write_to_file('\n### 0 HOP ###')
hits = ttp.evaluation_single_hop(gt)
patricks_log.save_time_since_start()

##############################################################################################################
0 HOP
start
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 10052/10052 [00:03<00:00, 3298.34it/s]


{'fn': 23453, 'fp': 1, 'tn': 549288, 'tp': 1596}
TP: 1596, FP: 1, TN: 549288, FN: 23453
Precision:  0.9993738259235442
Recall:  0.06371511836799847
F-Score:  0.11979283943930494
Accuracy:  0.9591634194498709
Gesamtzeit seit beginn: 0 Stunden, 41 Minute, 48 Sekunden, 0.0 Millisekunden, 714.0 Mikrosekunden, 255.5713653564453 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 0 Minute, 3 Sekunden, 0.0 Millisekunden, 763.0 Mikrosekunden, 808.7272644042969 Nanosekunden


In [14]:
print("##############################################################################################################")
print("1 HOP")
patricks_log.write_to_file('\n### 1 HOP ###')
hits = ttp.evaluation(gt, 1)
patricks_log.save_time_since_start()

##############################################################################################################
1 HOP
start
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 10052/10052 [04:10<00:00, 40.14it/s]


{'fn': 21121, 'fp': 1, 'tn': 549288, 'tp': 3928}
TP: 3928, FP: 1, TN: 549288, FN: 21121
Precision:  0.9997454823109951
Recall:  0.1568126472114649
F-Score:  0.2711022154503654
Accuracy:  0.9632237462957353
Gesamtzeit seit beginn: 0 Stunden, 45 Minute, 59 Sekunden, 0.0 Millisekunden, 867.0 Mikrosekunden, 935.8959197998047 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 4 Minute, 11 Sekunden, 0.0 Millisekunden, 143.0 Mikrosekunden, 897.0565795898438 Nanosekunden


In [15]:
print("##############################################################################################################")
print("2 HOP")
patricks_log.write_to_file('\n### 2 HOP ###')
hits = ttp.evaluation(gt, 2)
patricks_log.save_time_since_start()

##############################################################################################################
2 HOP
start
{'tn': 574338}
set every node in gt to fn
{'fn': 25049, 'tn': 549289}


100%|██████████| 10052/10052 [07:22<00:00, 22.70it/s]


{'fn': 1563, 'fp': 1, 'tn': 549288, 'tp': 23486}
TP: 23486, FP: 1, TN: 549288, FN: 1563
Precision:  0.9999574232554137
Recall:  0.93760229949299
F-Score:  0.9677764957469821
Accuracy:  0.9972768648426537
Gesamtzeit seit beginn: 0 Stunden, 53 Minute, 23 Sekunden, 0.0 Millisekunden, 336.0 Mikrosekunden, 809.8735809326172 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 7 Minute, 23 Sekunden, 0.0 Millisekunden, 485.0 Mikrosekunden, 439.06211853027344 Nanosekunden


In [16]:
print("##############################################################################################################")
print("Detection insights")
x = ttp.get_detection_insights()
patricks_log.save_time_since_start()

##############################################################################################################
Detection insights
Get Detection Insights
Unique Count of data_flow.y:  (tensor([0, 1, 2, 3, 4, 5]), tensor([133448,  43080, 374521,  12657,  10510,    122]))
Unique Count of pred: (tensor([False,  True]), tensor([564286,  10052]))
Unique Count of filtered_tensor: (tensor([0, 1, 2, 3, 4, 5]), tensor([  59, 1993, 4287, 1691, 1971,   51]))
Gesamtzeit seit beginn: 0 Stunden, 53 Minute, 23 Sekunden, 0.0 Millisekunden, 573.0 Mikrosekunden, 105.33523559570312 Nanosekunden
Zeit seit letztem Check: 0 Stunden, 0 Minute, 0 Sekunden, 0.0 Millisekunden, 216.0 Mikrosekunden, 747.283935546875 Nanosekunden
